# Importing Data

In [4]:
import pandas as pd
import numpy as np
import pprint as pp

In [5]:
FILE_PATH = '/content/drive/MyDrive/01 - Iniciação Científica/02 - Datasets/csv_files/EN2_STAR_CHR_0102890318_20070206T133547_20070402T070302.csv'

In [6]:
df = pd.read_csv(FILE_PATH)
df.head()

,DATEBARTT,WHITEFLUXSYS
0,54138.073885,219929.30
1,54138.079811,220816.39
2,54138.085737,220129.64
3,54138.091662,219876.34
4,54138.097588,219744.33


In [7]:
y = df.WHITEFLUXSYS.to_numpy()
x = df.DATEBARTT.to_numpy()

# Data normalization

Before applying Butterworth filter, we have to ...

## 1. Artificial borders

In [8]:
def artifical_borders(array, num):
  aux_pre = np.zeros(num)
  aux_pos = np.zeros(num)
  i = 0
  for i in range(num):
    aux_pre[i] = array[0]
    aux_pos[i] = array[-1]
  
  return np.concatenate((aux_pre, array, aux_pos)).ravel()

## 2. Multiplying data by (-1)^i

In [9]:
def multiplying_by_minus_one_to_index(array):
  i = 0
  new_array = np.ones(len(array))

  for i in range(len(array)):
    new_array[i] = array[i] * ( (-1)**(i) )
  
  return new_array

## 3. Padding

In [10]:
def padding(array):
  return np.append(array, np.zeros(len(array)))

Now, our date is ready to be filtered.

The first step is to calculate the Fourier Transform.

# Fourier Transform


In [11]:
def fourier_transform(array):
  return np.fft.fft(array)

# Butterworth Filter

# Inverse Fourier Transform

In [12]:
def inverse_fourier_transform(array):
  return np.fft.ifft(array)

# Undoing 2. Multiplying data by (-1)^i

In [13]:
y1_pos = multiplying_by_minus_one_to_index( ___ )

In [22]:
x1_pos = [1, 2] # ultimo x

# Undoing 3. Padding

In [14]:
y2_pos = y1_pos[:int(len(y1_pos)/2)]

In [23]:
x2_pos = x1_pos[:int(len(x1_pos)/2)]

# Undoing 1. Artificial borders

In [15]:
param = int(15)

In [16]:
y3_pos = np.delete(y2_pos, np.s_[:param])
y4_pos = np.delete(y3_pos, np.s_[-param:])

In [24]:
x3_pos = np.delete(x2_pos, np.s_[:param])
x4_pos = np.delete(x3_pos, np.s_[-param:])